In [1]:
# !pip install -e .

In [1]:
import pandas as pd 

from deeplens.extractor import ExtractSingleSample
from deeplens.intervene import InterveneFeatures, ReinjectSingleSample
from deeplens.utils.analysis import plot_topk_distribution, get_top_k_tokens

In [2]:
def pipeline(
        text,
        hf_model: str = "gpt2",
        layer: int = 3,
        sae_model: str = None,
        sae_config: str = None,
        feature: int = -1,
        alpha: float = 5.0,
        tok_position: int = -1,
        generate: bool = False,
        max_new_tokens: int = 25,
        temperature: float = 1.0
    ):
    # Initialize the classes
    extractor = ExtractSingleSample(hf_model=hf_model, layer=layer)
    intervene = InterveneFeatures(sae_model=sae_model, sae_config=sae_config)
    reinject = ReinjectSingleSample(hf_model=hf_model)
    
    # Extract the activations
    acts = extractor.get_mlp_acts(text)
    # Get alive features from the extracted activations
    features = intervene.get_alive_features(acts, token_position=tok_position)
    print(f"{len(features)} alive features discovered at position {tok_position}.")
    # intervene on the features and return the raw original activations, 
    # SAE-decoded baseline activations, and modified activations for the 
    # selected token position for the selected token position.
    print(f"Modifying feature {features[feature].item()}")
    original, original_decoded, modified_decoded = intervene.intervene_feature(
        activations=acts, 
        feature=features[feature].item(),
        alpha=alpha, 
        token_positions=tok_position
    )
    # For each set of MLP activations, compute the output logits
    feats = [original, original_decoded, modified_decoded]
    outputs = []
    for f in feats:
        out = reinject.reinject_and_generate(
            text=text,
            modified_activations=f,
            layer=layer,
            generate=generate,
            max_new_tokens=max_new_tokens,
            temperature=temperature
        )
        outputs.append(out)

    return tuple(outputs)

In [5]:
original, original_output, modified_output = pipeline(
    text="He said he would",
    hf_model="gpt2",
    layer=3,
    sae_model=r"C:\code\deeplens\demo\model\best_model.pt",
    sae_config=r"C:\code\deeplens\demo\config.yaml",
    feature=-1,
    alpha=1000.0,
    tok_position=-1,
    generate=False,
    temperature=1.0
)

Running on device: cuda
Running on device: cuda
Running on device: cuda
591 alive features discovered at position -1.
Modifying feature 24524


In [ ]:
plot_topk_distribution(
    original, 
    use_softmax=True, 
    k=50, 
    position=-1, 
    title="Raw (no SAE)"
)

plot_topk_distribution(
    original_output, 
    use_softmax=True, 
    k=50, 
    position=-1, 
    title="SAE Baseline"
)

plot_topk_distribution(
    modified_output, 
    use_softmax=True, 
    k=50, 
    position=-1, 
    title="SAE + Intervention"
)

In [7]:
results = get_top_k_tokens(
    modified_output,
    k=5,
    tokenizer="gpt2",
    verbose=False
)

In [ ]:
import ollama
from ollama._client import Client

def auto_interpret():
    try:
        client = Client()
    except Exception as e:
        print(f"An error ocurred: {e}")

In [ ]:
from deeplens.extractor import ExtractSingleSample
from deeplens.intervene import InterveneFeatures

sentences = [
    "What did you do?",
    "What have you done?",
    "What we did was amazing",
    "What came after was unexpected"
]


def get_most_active_features(sentences, hf_model: str = "gpt2", layer: int = 3, k: int | None = None):
    mlp_extractor = ExtractSingleSample(hf_model=hf_model, layer=layer)
    sae_features_extractor = InterveneFeatures(
        sae_model=r"C:\code\deeplens\demo\model\best_model.pt", 
        sae_config=r"C:\code\deeplens\demo\config.yaml"
    )

    features = []
    for sent in sentences:
        acts = mlp_extractor.get_mlp_acts(sample=sent)
        for i in range(acts.shape[0]):
            feats = sae_features_extractor.get_alive_features(activations=acts, token_position=i, k=k)
            features.append(feats.cpu())

    return features

feats = get_most_active_features(sentences, k=5)
feats